In [1]:
import requests
import json
# from bs4 import BeautifulSoup
import time
import pickle

In [2]:
token = 'E9LQF4KUV7SE2MH0'

# Finding Total Accounts

In [24]:
total_accounts = 9022265 # last update
# finding total number of hon international accounts using binary search
lo, hi = total_accounts, 9100000
while lo+1 < hi:
    mid = (lo+hi)//2
    r = requests.get('http://api.heroesofnewerth.com/player_statistics/ranked/accountid/{}/?token={}'.format(mid, token))
    if r.text == 'Unauthorized!':
        print(r.text)
        break
    try:
        player_dict = r.json()
        lo = mid
    except ValueError:
        hi = mid-1
    print(lo, hi)
print('total accounts:', lo)
total_accounts = lo

9022265 9061131
9022265 9041697
9022265 9031980
9022265 9027121
9022265 9024692
9022265 9023477
9022265 9022870
9022265 9022566
9022265 9022414
9022265 9022338
9022265 9022300
9022265 9022281
9022265 9022272
9022265 9022267
9022266 9022267
total accounts: 9022266


# Finding match statistics

In [41]:
db_file = 'hon_matches.pkl'
try:
    with open(db_file, 'rb') as f:
        all_matches = pickle.load(f)
except FileNotFoundError:
    all_matches = dict()
print(len(all_matches))

54345


In [42]:
def clean(accounts):
    new_accounts = []
    removed = 0
    for acc in accounts:
        secs = int(acc['secs'])
        if int(acc['actions']) <= 300: # if APM is too low, the player did not contribute to the team, (disconnection assumed)
            removed += 1
        else:
            new_acc = dict()
            new_acc['hero_id'] = int(acc['hero_id'])
            new_acc['win'] = int(acc['wins'])
            new_acc['concede'] = int(acc['concedes'])
            new_acc['match_id'] = int(acc['match_id'])
            new_acc['secs'] = secs
            new_acc['team'] = int(acc['team']) - 1 # legion 0, hellbourne 1
            new_accounts.append(new_acc)
    if removed:
        print('Removed Low APM:', removed)
    return new_accounts

In [43]:
# return a dict that can be accessed by [match_id]
def aggregate(accounts):
    matches = dict()
    for acc in accounts:
        matchid = acc['match_id'] 
        if matchid not in matches:
            matches[matchid] = {
                'concedes': 0,
                'secs': 0,
                'legion': list(),
                'hellbourne': list()
            }
        # use max because secs is the amount of seconds that player is in the game
        matches[matchid]['secs'] = max(matches[matchid]['secs'], acc['secs'])
        if acc['secs'] == matches[matchid]['secs']: # this guy plays until the end of the game
            matches[matchid]['winner'] = acc['team'] if acc['win'] else 1 - acc['team']
        if acc['concede']:
            matches[matchid]['concedes'] += 1
        if acc['team'] == 0:
            matches[matchid]['legion'].append(acc['hero_id'])
        else:
            matches[matchid]['hellbourne'].append(acc['hero_id'])
    return matches

In [63]:
matchid = 147862153 # latest match id found

In [70]:
import sys
dump_every = 10 # iterations
sleep_period = 0.5 # sleep to prevent API requests limit
for iter in range(2501):
    matchids = []
    for i in range(25):
        matchids.append(str(matchid))
        matchid -= 1
    url = 'http://api.heroesofnewerth.com/multi_match/statistics/matchids/{}/?token={}'.format('+'.join(matchids), token)
    print('-- Requesting...', end=' ')
    while True:
        try:
            r = requests.get(url)
            break
        except:
            print('ConnectionError, pause for {} secs'.format(sleep_period * 20), file=sys.stderr)
            time.sleep(sleep_period * 20)
    print('Sleeping...', end=' ')
    time.sleep(sleep_period)
    if r.text == 'No results.':
        print(r.text)
        continue
    try:
        print('Decoding...', end=' ')
        match_accounts = r.json()
    except ValueError:
        print('Error:', r.text, file=sys.stderr)
        time.sleep(sleep_period * 10)
        continue
    match_accounts = clean(match_accounts)
    matches = aggregate(match_accounts)
    all_matches.update(matches)
    print('Total matches in this/all batch:', len(matches), len(all_matches))
    # print('Next Unused MatchID:', matchid)
    if iter % dump_every == 0:
        print('==> Dumping at iter', iter)
        with open(db_file, 'wb') as f:
            pickle.dump(all_matches, f)

-- Requesting... Sleeping... Decoding... Removed Low APM: 1
Total matches in this/all batch: 14 60487
==> Dumping at iter 0
-- Requesting... Sleeping... Decoding... Removed Low APM: 4
Total matches in this/all batch: 14 60501
-- Requesting... Sleeping... Decoding... Removed Low APM: 2
Total matches in this/all batch: 16 60517
-- Requesting... Sleeping... Decoding... Removed Low APM: 1
Total matches in this/all batch: 17 60534
-- Requesting... Sleeping... Decoding... Total matches in this/all batch: 14 60548
-- Requesting... Sleeping... Decoding... Removed Low APM: 2
Total matches in this/all batch: 16 60564
-- Requesting... Sleeping... Decoding... Removed Low APM: 3
Total matches in this/all batch: 14 60578
-- Requesting... Sleeping... Decoding... Removed Low APM: 3
Total matches in this/all batch: 16 60594
-- Requesting... Sleeping... Decoding... Removed Low APM: 2
Total matches in this/all batch: 17 60611
-- Requesting... Sleeping... Decoding... Removed Low APM: 1
Total matches in th

ConnectionError, pause for 10.0 secs


Sleeping... Decoding... Removed Low APM: 1
Total matches in this/all batch: 16 60656
-- Requesting... Sleeping... Decoding... Removed Low APM: 1
Total matches in this/all batch: 18 60674
-- Requesting... Sleeping... Decoding... Removed Low APM: 2
Total matches in this/all batch: 15 60689
-- Requesting... Sleeping... Decoding... Total matches in this/all batch: 16 60705
-- Requesting... Sleeping... Decoding... Removed Low APM: 1
Total matches in this/all batch: 16 60721
-- Requesting... Sleeping... Decoding... Total matches in this/all batch: 13 60734
-- Requesting... Sleeping... Decoding... Removed Low APM: 1
Total matches in this/all batch: 15 60749
-- Requesting... Sleeping... Decoding... Total matches in this/all batch: 15 60764
-- Requesting... Sleeping... Decoding... Total matches in this/all batch: 13 60777
-- Requesting... Sleeping... Decoding... Removed Low APM: 2
Total matches in this/all batch: 18 60795
==> Dumping at iter 20
-- Requesting... Sleeping... Decoding... Removed L

ConnectionError, pause for 10.0 secs


Sleeping... Decoding... Removed Low APM: 3
Total matches in this/all batch: 15 62540
-- Requesting... Sleeping... Decoding... Removed Low APM: 3
Total matches in this/all batch: 17 62557
-- Requesting... Sleeping... Decoding... Removed Low APM: 4
Total matches in this/all batch: 12 62569
-- Requesting... Sleeping... Decoding... Removed Low APM: 5
Total matches in this/all batch: 17 62586
-- Requesting... Sleeping... Decoding... Removed Low APM: 1
Total matches in this/all batch: 18 62604
-- Requesting... Sleeping... Decoding... Removed Low APM: 4
Total matches in this/all batch: 14 62618
-- Requesting... Sleeping... Decoding... Removed Low APM: 4
Total matches in this/all batch: 15 62633
-- Requesting... Sleeping... Decoding... Removed Low APM: 2
Total matches in this/all batch: 17 62650
-- Requesting... Sleeping... Decoding... Removed Low APM: 2
Total matches in this/all batch: 18 62668
-- Requesting... Sleeping... Decoding... Removed Low APM: 5
Total matches in this/all batch: 16 626

...

Sleeping... Decoding... Removed Low APM: 2
Total matches in this/all batch: 14 86462
-- Requesting... 

ConnectionError, pause for 10.0 secs


Sleeping... Decoding... Total matches in this/all batch: 18 86480
-- Requesting... Sleeping... Decoding... Removed Low APM: 4
Total matches in this/all batch: 17 86497
-- Requesting... Sleeping... Decoding... Removed Low APM: 1
Total matches in this/all batch: 17 86514
-- Requesting... Sleeping... Decoding... Removed Low APM: 5
Total matches in this/all batch: 16 86530
-- Requesting... Sleeping... Decoding... Total matches in this/all batch: 14 86544
-- Requesting... Sleeping... Decoding... Total matches in this/all batch: 16 86560
-- Requesting... Sleeping... Decoding... Removed Low APM: 1
Total matches in this/all batch: 15 86575
-- Requesting... Sleeping... Decoding... Removed Low APM: 2
Total matches in this/all batch: 19 86594
==> Dumping at iter 1620
-- Requesting... Sleeping... Decoding... Total matches in this/all batch: 18 86612
-- Requesting... Sleeping... Decoding... Removed Low APM: 1
Total matches in this/all batch: 14 86626
-- Requesting... Sleeping... Decoding... Removed

ConnectionError, pause for 10.0 secs


Sleeping... Decoding... Removed Low APM: 1
Total matches in this/all batch: 16 93242
-- Requesting... Sleeping... Decoding... Removed Low APM: 2
Total matches in this/all batch: 19 93261
-- Requesting... Sleeping... Decoding... Removed Low APM: 1
Total matches in this/all batch: 14 93275
-- Requesting... Sleeping... Decoding... Total matches in this/all batch: 13 93288
-- Requesting... Sleeping... Decoding... Removed Low APM: 2
Total matches in this/all batch: 15 93303
-- Requesting... Sleeping... Decoding... Removed Low APM: 2
Total matches in this/all batch: 17 93320
-- Requesting... Sleeping... Decoding... Total matches in this/all batch: 15 93335
-- Requesting... Sleeping... Decoding... Removed Low APM: 2
Total matches in this/all batch: 15 93350
==> Dumping at iter 2040
-- Requesting... Sleeping... Decoding... Removed Low APM: 3
Total matches in this/all batch: 18 93368
-- Requesting... Sleeping... Decoding... Removed Low APM: 3
Total matches in this/all batch: 18 93386
-- Request

In [71]:
print('Next Unused MatchID:', matchid)

Next Unused MatchID: 147790103


In [72]:
match_accounts

[{'concede': 1,
  'hero_id': 38,
  'match_id': 147790104,
  'secs': 1496,
  'team': 1,
  'win': 0},
 {'concede': 0,
  'hero_id': 10,
  'match_id': 147790104,
  'secs': 1496,
  'team': 0,
  'win': 1},
 {'concede': 1,
  'hero_id': 167,
  'match_id': 147790104,
  'secs': 1496,
  'team': 1,
  'win': 0},
 {'concede': 1,
  'hero_id': 20,
  'match_id': 147790104,
  'secs': 543,
  'team': 1,
  'win': 0},
 {'concede': 1,
  'hero_id': 109,
  'match_id': 147790104,
  'secs': 1496,
  'team': 1,
  'win': 0},
 {'concede': 0,
  'hero_id': 7,
  'match_id': 147790104,
  'secs': 1496,
  'team': 0,
  'win': 1},
 {'concede': 0,
  'hero_id': 5,
  'match_id': 147790104,
  'secs': 1496,
  'team': 0,
  'win': 1},
 {'concede': 1,
  'hero_id': 43,
  'match_id': 147790104,
  'secs': 1496,
  'team': 1,
  'win': 0},
 {'concede': 0,
  'hero_id': 41,
  'match_id': 147790104,
  'secs': 1496,
  'team': 0,
  'win': 1},
 {'concede': 0,
  'hero_id': 196,
  'match_id': 147790104,
  'secs': 1496,
  'team': 0,
  'win': 1},


In [73]:
matches

{147790104: {'concedes': 5,
  'hellbourne': [38, 167, 20, 109, 43],
  'legion': [10, 7, 5, 41, 196],
  'secs': 1496,
  'winner': 0},
 147790105: {'concedes': 0,
  'hellbourne': [163, 241, 249, 245, 2],
  'legion': [2, 2, 125, 162, 104],
  'secs': 727,
  'winner': 0},
 147790106: {'concedes': 5,
  'hellbourne': [242, 6, 43, 194, 20],
  'legion': [164, 161, 218, 216, 249],
  'secs': 1597,
  'winner': 1},
 147790107: {'concedes': 5,
  'hellbourne': [207, 27, 96, 117, 197],
  'legion': [211, 38, 40, 195, 3],
  'secs': 1898,
  'winner': 0},
 147790108: {'concedes': 0,
  'hellbourne': [12, 94, 163, 225, 228],
  'legion': [170, 117, 196, 110, 207],
  'secs': 4476,
  'winner': 0},
 147790110: {'concedes': 5,
  'hellbourne': [38, 236, 249, 16, 43],
  'legion': [235, 17, 221, 216, 167],
  'secs': 1605,
  'winner': 1},
 147790111: {'concedes': 5,
  'hellbourne': [109, 226, 102, 6, 228],
  'legion': [103, 205, 218, 247, 164],
  'secs': 2403,
  'winner': 1},
 147790114: {'concedes': 5,
  'hellbourn

In [74]:
with open(db_file, 'wb') as f:
    pickle.dump(all_matches, f)

# Heroes reference
http://api.heroesofnewerth.com/heroes/all?token=E9LQF4KUV7SE2MH0

In [208]:
r = requests.get('http://api.heroesofnewerth.com/heroes/all?token=E9LQF4KUV7SE2MH0')

In [210]:
heroes_dict = r.json()
heroes_dict

{'Hero_Accursed': {'40': {'attacktype': 'melee',
   'description': 'Many have speculated as to what great, damned warrior now lives in torment within the burning armor that marches alongside the Hellbourne army. The Accursed, as he is known, screams in agony as he brings fiery devastation to his foes, but his actions seem driven more by a desire to find oblivion than by malice.',
   'disp_name': 'Accursed',
   'hero_id': '40',
   'primaryattribute': 'Strength',
   'team': 'Hellbourne'}},
 'Hero_Adrenaline': {'249': {'attacktype': 'ranged',
   'description': 'Adrenaline is the physical embodiment of the chaos and fury that rages within the ancient heart of the True Evil. His sole purpose is to lead The Third Corruption and carve a bloody swath through daemon, man, beast, and god, clearing the way for Apex to establish his presence and begin his final conquest of the known realms.',
   'disp_name': 'Adrenaline',
   'hero_id': '249',
   'primaryattribute': 'Agility',
   'team': 'Hellbourn

In [211]:
len(heroes_dict)

134

In [232]:
new_heroes_dict = dict()
for value in heroes_dict.values():
    hero_id, hero = list(value.items())[0]
    hero_id = int(hero_id)
    new_heroes_dict[hero_id] = hero
new_heroes_dict

{2: {'attacktype': 'melee',
  'description': 'A stolid and stalwart defender of the weak, whether Man or Beast, the long-lived Armadon refused to take part in the war between the Legion and the Horde, and instead maintained a neutral haven amidst the strife.  With the arrival of the Hellbourne, however, Armadon&#039;s choice became clear, and he hefted his mighty mace in the service of good.  The gruff warrior is now a key hero in the New Legion.',
  'disp_name': 'Armadon',
  'hero_id': '2',
  'primaryattribute': 'Strength',
  'team': 'Legion'},
 3: {'attacktype': 'melee',
  'description': 'Earth-shakers and wall-breakers, Behemoths march across the battlefield wielding uprooted trees with deadly force.  The sheer force of their attacks brings enemies to their knees, where powerful blows make short work of them. Of all the Beasts, none are so mighty as these noble creatures, nor any so willing to stand between the daemons and their friends.',
  'disp_name': 'Behemoth',
  'hero_id': '3'

In [234]:
with open('heroes_name.pkl', 'wb') as f:
    pickle.dump(new_heroes_dict, f)